# Analysis 2e6 muons by CJ - ROI cut
In this notebook, we will see the last data from CJ. Only the entries within the ROI are considered. The ROI is defined as follows:
- `x`, `y` in `[-500, +500]`
- `z` in `[-1000, +1000]`

The file after the cut has `9324212` entries and `11077` events.

In [ ]:
import pandas as pd
import os

dir_in = os.path.join("..", "Data", "cutROI")
#files = ["output2eROI_part1.csv", "output2eROI_part2.csv", "output2eROI_part3.csv", "output2eROI_part4.csv", "output2eROI_part5.csv"]
#df = pd.read_csv(os.path.join(dir_in, files[0]), index_col=False)
#for file in files[1:]:
#    dfi = pd.read_csv(os.path.join(dir_in, file), index_col=False)
#    df = pd.concat([df, dfi])
#df.head()

new_file = "output2eROI_all_wt_ops.csv"
opyield = 40
df = pd.read_csv(os.path.join(dir_in, new_file), index_col=False)
df.columns

In [ ]:
df = df[df.energydeposition > 0]
df["OPcounter"] = df.energydeposition * op_yield * df.OPs

op_yield = 40 # 40 op /kev
dfdet = df[df.OPcounter > 0]
print("[Info] Number of entries: {}".format(df.shape[0]))
print("[Info] Number of distinct events: {}".format(df.eventnumber.nunique()))
print("[Info] Number of detected entries: {}".format(dfdet.shape[0]))
print("[Info] Number of detected events: {}".format(dfdet.eventnumber.nunique()))
df.head()

In [ ]:
edep_gb = df.groupby("eventnumber").energydeposition.sum().reset_index()
edep_muons_gb = df[df.PID==13].groupby("eventnumber").energydeposition.sum().reset_index()
edep_electrons_gamma_gb = df[(df.PID.isin([11, 22]))].groupby("eventnumber").energydeposition.sum().reset_index()
stats = edep_gb.energydeposition.describe()
stats_muons = edep_muons_gb.energydeposition.describe()
stats_electron_gamma = edep_electrons_gamma_gb.energydeposition.describe()

pd.DataFrame([stats, stats_muons, stats_electron_gamma], index=['all', 'muons', 'e- or gammas'])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = [25, 15]
plt.rcParams.update({'font.size': 18})

In [ ]:
label_template = "number events: {:}\n" \
                 "min: {:.2f} MeV, max: {:.2f} MeV\n" \
                 "mean: {:.2f} MeV, std: {:.2f}"

label_all = label_template.format(stats.loc["count"], stats.loc["min"]/1000, stats.loc["max"]/1000, 
                                  stats.loc["mean"]/1000, stats.loc["std"])
label_muon = label_template.format(stats_muons.loc["count"], stats_muons.loc["min"]/1000, stats_muons.loc["max"]/1000, 
                                   stats_muons.loc["mean"]/1000, stats_muons.loc["std"])
label_egam = label_template.format(stats_electron_gamma.loc["count"], stats_electron_gamma.loc["min"]/1000, stats_electron_gamma.loc["max"]/1000,
                                   stats_electron_gamma.loc["mean"]/1000, stats_electron_gamma.loc["std"])

bins = np.logspace(0, 9, 100)
plt.subplot(3, 1, 1)
plt.hist(edep_gb.energydeposition, bins, label="All entries", alpha=0.5, color='b')
plt.title("Cumulative Edep Distribution in ROI")
plt.xlabel("Cumulative Edep (KeV)")
plt.ylabel("count(#entries)")
plt.xscale('log')
plt.ylim(0, 1000)
plt.legend()
plt.text(10, 600, label_all)


plt.subplot(3, 1, 2)
plt.hist(edep_muons_gb.energydeposition, bins, label="Muon entries", alpha=0.5, color='orange')
plt.xlabel("Cumulative Edep (KeV)")
plt.ylabel("count(#entries)")
plt.xscale('log')
plt.ylim(0, 1000)
plt.legend()
plt.text(10, 600, label_muon)

plt.subplot(3, 1, 3)
plt.hist(edep_electrons_gamma_gb.energydeposition, bins, label="e- or gamma entries", alpha=0.5, color='g')
plt.xlabel("Cumulative Edep (KeV)")
plt.ylabel("count(#entries)")
plt.xscale('log')
plt.ylim(0, 1000)
plt.legend()
plt.text(10, 400, label_egam)

plt.show()

The above plots reveal interesting thing about energy deposition distribution.

1. **first plot**: we consider the scenario in which *all the entries* with Edep>0 deposit in LAr and then potentially generate scintillation
1. **second plot**: we consider only the energy deposition by *muon entries*. If the Edep consider also the energy passed to other particles, then this entries are not meaningful for out purposes.
1. **third plot**: we consider only the energy depositions by *electrons and gammas* particles. This is the scenario in which the Edep field consider both the deposited energy and the energy passed to other particles. In this case, the only particle that lead to scintillation are electrons and gammas.

# Analysis OP counter

In [ ]:
op_yield = 40
df["OPcounter"] = df.energydeposition * op_yield * df.OPs

opcounter_gb = df.groupby("eventnumber").OPcounter.sum().reset_index()
stats_opcounter = opcounter_gb.OPcounter.describe()
stats_opcounter

In [ ]:
label = "min: {:.2f}, max: {:.2f}\n".format(stats_opcounter.loc["min"], stats_opcounter.loc["max"]/1000)
label += "mean: {:.2f}, std: {:.2f}\n".format(stats_opcounter.loc["mean"]/1000, stats_opcounter.loc["std"])

bins = np.logspace(0, 9, 100)

# histo edep
plt.hist(edep_gb.energydeposition, bins, alpha=0.7, label="Cum.Edep")
# histo op counter
plt.hist(opcounter_gb.OPcounter, bins, alpha=0.7, label="#OPs Detected")

# layout
plt.title("Comparison Cum.Edep vs OP Detected")
plt.xlabel("Cumulative Edep (KeV) or Number OPs Detected")
plt.ylabel("count(#entries)")
plt.xscale('log')
plt.legend()
plt.show()